Behavioral Cloning P3
--

In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Cropping2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import load_model
import csv
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from random import shuffle

Using TensorFlow backend.


In [2]:
angle_adjust = [0, 0.2, -0.2]
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                # center, left, right
                for i in range(3):
                    name = './IMG/' + batch_sample[i].split('/')[-1]
                    image = cv2.imread(name)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    angle = float(batch_sample[3]) + angle_adjust[i]
                    images.append(image)
                    angles.append(angle)
                    # flip image and angle
                    image_flipped = np.fliplr(image)
                    angle_flipped = -angle
                    images.append(image_flipped)
                    angles.append(angle_flipped)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [16]:
samples = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

BATCH_SIZE = 64
NB_EPOCH = 3

train_samples, validation_samples = train_test_split(samples, test_size=0.2)
train_generator = generator(train_samples, batch_size=BATCH_SIZE)
validation_generator = generator(validation_samples, batch_size=BATCH_SIZE)

In [17]:
model = Sequential()
# preprocessing
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: x / 255, input_shape=(160,320,3)))

# https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/
# conv1 3@90x320
model.add(Convolution2D(nb_filter=24, nb_row=5, nb_col=5, subsample=(2,2), border_mode='valid'))
# conv2 24@43x158
model.add(Convolution2D(nb_filter=36, nb_row=5, nb_col=5, subsample=(2,2), border_mode='valid'))
# conv3 36@20x77
model.add(Convolution2D(nb_filter=48, nb_row=5, nb_col=5, subsample=(2,2), border_mode='valid'))
# conv4 48@8x37
model.add(Convolution2D(nb_filter=64, nb_row=3, nb_col=3, subsample=(1,1), border_mode='valid'))
# conv5
model.add(Convolution2D(nb_filter=64, nb_row=3, nb_col=3, subsample=(1,1), border_mode='valid'))
model.add(Flatten())

# FC1
model.add(Dense(100))
model.add(Activation('relu'))
# FC2
model.add(Dense(50))
model.add(Activation('relu'))
# FC3
model.add(Dense(10))
model.add(Activation('relu'))
# Output
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
history = model.fit_generator(train_generator,
        samples_per_epoch=len(train_samples),
        validation_data=validation_generator,
        nb_val_samples=len(validation_samples),
        nb_epoch=NB_EPOCH)
model.save('model.h5')
print('model saved.')

Epoch 1/3
4992/5093 [============================>.] - ETA: 0s - loss: 0.1875

KeyboardInterrupt: 

In [5]:
print(model.summary())
print(history.history)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_1 (Cropping2D)        (None, 90, 320, 3)    0           cropping2d_input_1[0][0]         
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 90, 320, 3)    0           cropping2d_1[0][0]               
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 43, 158, 24)   1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 20, 77, 36)    21636       convolution2d_1[0][0]            
___________________________________________________________________________________________

In [ ]:
model = Sequential()
model = load_model('model.h5')